In [1]:
import numpy as np
import pandas as pd
import subprocess as sub

## step 1. Generate sample set by metropolis method

In [77]:
sub.run(['g++', 'step1_plaquette_r.cpp', '-o', 'step1', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib', '-ltrng4', '-std=gnu++11'])
sub.run(['./step1'])
print("step1 finished\n")

#Data of Energy&nn in fileout.txt
f = pd.read_csv('fileout.txt', sep = ' ', header = 0)
temp = f.temp[0]
J0 = np.array(f.ene)
J1 = np.array(f.nn)
J2 = np.array(f.nnn)
J3 = np.array(f.nnnn)
num = len(J0)
nth = int(f.nth[0]) #How much consideration of nearest neighbors
Jmat = np.array([J1, J2, J3]).T
target_mat = np.hstack([np.ones((num, 1)), Jmat[:, 0:nth]])
Jlist, err, _, _ = np.linalg.lstsq(target_mat, J0, rcond = None)
print("T =", temp)
print("Energy, Jlist =", Jlist)
print("error =", err)

#Data of coefficient in filein.txt
with open("filein.txt", "w") as file:
    file.write('%d\n' %nth)
    for i in range (nth + 1):
        data = Jlist[i]
        file.write('%.10g\n' % data)
    file.write('%.10g' %temp)

step1 finished

T = 4.493
Energy, Jlist = [ 1.33668245  1.09433981 -0.01969749]
error = [31651.93704524]


## step 2. optimize J, regenerate sample set by wolff method

In [78]:
sub.run(['g++', 'step2_nnnwolff.cpp', '-o', 'step2', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib','-ltrng4', '-std=gnu++11'])
for i in range (10):
    sub.run(['./step2'])

    #Data of Energy&nn in fileout.txt
    f = pd.read_csv('fileout.txt', sep = ' ', header = 0)
    temp = f.temp[0]
    J0 = np.array(f.ene)
    J1 = np.array(f.nn)
    J2 = np.array(f.nnn)
    J3 = np.array(f.nnnn)
    num = len(J0)
    nth = int(f.nth[0]) #How much consideration of nearest neighbors
    Jmat = np.array([J1, J2, J3]).T
    target_mat = np.hstack([np.ones((num, 1)), Jmat[:, 0:nth]])
    Jlist, err, _, _ = np.linalg.lstsq(target_mat, J0, rcond = None)
    print("T =", temp, " Energy, J =", Jlist, " error =", err)
    #Data of coefficient in filein.txt
    with open("filein.txt", "w") as file:
        file.write('%d\n' %nth)
        for i in range (nth + 1):
            data = Jlist[i]
            file.write('%.10g\n' % data)
        if temp>2.5: file.write('%.10g' %temp)
        else: file.write('%.10g' %(temp+0.2))

T = 4.293  Energy, J = [ 1.55089393  1.09939139 -0.02175871]  error = [30406.42589704]
T = 4.093  Energy, J = [ 1.67342522  1.10469988 -0.02520128]  error = [28990.74985843]
T = 3.8930000000000002  Energy, J = [ 1.70594633  1.10472465 -0.02343728]  error = [28916.05999024]
T = 3.693  Energy, J = [ 2.3678694   1.11834624 -0.02875267]  error = [26871.54736191]
T = 3.4930000000000003  Energy, J = [ 2.65876026  1.12482118 -0.03244802]  error = [26888.46755139]
T = 3.293  Energy, J = [ 2.78360792  1.12729474 -0.03306185]  error = [25085.37445122]
T = 3.093  Energy, J = [ 3.21536665  1.13562147 -0.03679098]  error = [22819.17140318]
T = 2.8930000000000002  Energy, J = [ 3.84904768  1.14499881 -0.04060215]  error = [21345.37054317]
T = 2.693  Energy, J = [ 4.39140639  1.15093251 -0.04202834]  error = [18327.51275042]
T = 2.4930000000000003  Energy, J = [ 5.00898833  1.15046581 -0.03660591]  error = [15520.45420131]


In [80]:
for i in range (10):
    sub.run(['./step2'])

    #Data of Energy&nn in fileout.txt
    f = pd.read_csv('fileout.txt', sep = ' ', header = 0)
    temp = f.temp[0]
    J0 = np.array(f.ene)
    J1 = np.array(f.nn)
    J2 = np.array(f.nnn)
    J3 = np.array(f.nnnn)
    num = len(J0)
    nth = int(f.nth[0]) #How much consideration of nearest neighbors
    Jmat = np.array([J1, J2, J3]).T
    target_mat = np.hstack([np.ones((num, 1)), Jmat[:, 0:nth]])
    Jlist, err, _, _ = np.linalg.lstsq(target_mat, J0, rcond = None)
    print("T =", temp, " Energy, J =", Jlist, " error =", err)
    #Data of coefficient in filein.txt
    with open("filein.txt", "w") as file:
        file.write('%d\n' %nth)
        for i in range (nth + 1):
            data = Jlist[i]
            file.write('%.10g\n' % data)
        file.write('%.10g' %(temp+0.2))

T = 2.4930000000000003  Energy, J = [ 9.72995309  1.24517261 -0.10021949]  error = [6707.0809996]
T = 2.4930000000000003  Energy, J = [2.4524962  1.0530221  0.02935609]  error = [28782.7347644]
T = 2.4930000000000003  Energy, J = [ 9.70253307  1.24601851 -0.10127187]  error = [6893.2650188]
T = 2.4930000000000003  Energy, J = [2.58840128 1.05495026 0.02893366]  error = [28105.77945283]
T = 2.4930000000000003  Energy, J = [ 9.59890405  1.24441355 -0.10028699]  error = [7048.16536798]
T = 2.4930000000000003  Energy, J = [2.95121445 1.05525641 0.03169247]  error = [28288.64713758]
T = 2.4930000000000003  Energy, J = [ 9.47404068  1.24230819 -0.09893224]  error = [6807.57436817]
T = 2.4930000000000003  Energy, J = [2.99289504 1.0580907  0.02979729]  error = [28173.65477291]
T = 2.4930000000000003  Energy, J = [ 9.43924951  1.24318457 -0.10006869]  error = [7514.22672411]
T = 2.4930000000000003  Energy, J = [2.5956543  1.05411902 0.02996123]  error = [28762.90225709]


## step 3. Compare magnetization

In [4]:
sub.run(['g++', 'step3_nnnwolff_Mplot.cpp', '-o', 'step3', '-I/home/hyejin/trng4-4.22', '-L/opt/trng/lib', '-ltrng4', '-std=gnu++11'])
sub.run(['./step3'])
print("step3 finished")

KeyboardInterrupt: 